model can train with seasons () to ()
what are the features?


dataframe:
home_team | away_team | ranking_home_team | ranking_away_team || result
________________________________________________________________________

4 primeres columnes: features
5 columna: target

train (season tal to tal: info features + target)

test (apply to () with teh following features)

print:

features, target, predictions

In [ ]:
def preprocessing(input_df):    
    filtered_score_df = input_df[input_df['score'].notna()].copy()
    split_scores = filtered_score_df['score'].str.split(':', expand=True).astype(int)
    filtered_score_df['home_score'] = split_scores[0]
    filtered_score_df['away_score'] = split_scores[1]
    return filtered_score_df

def format_date(df):
        split_dates = df['date'].str.split('/', expand=True).astype(int)
        df_copy = df.copy()
        df_copy['day'] = split_dates[1]
        df_copy['month'] = split_dates[0]
        df_copy['year'] = split_dates[2]
        def correct_year(year):
            if year > 25:
                return year + 1900
            else:
                return year + 2000
        df_copy['year'] = df_copy['year'].apply(correct_year)
        df_copy['date1'] = pd.to_datetime(df_copy[['year', 'month', 'day']])
        return(df_copy)

df_filtered = preprocessing(df.copy())
def separate_wins_loses_ties(df_filtered):
    df_filtered['Win_home'] = np.where(df_filtered['home_score'] > df_filtered['away_score'], 1, 0)
    df_filtered['Win_away'] = np.where(df_filtered['home_score'] < df_filtered['away_score'], 1, 0)
    df_filtered['Ties'] = np.where(df_filtered['home_score'] == df_filtered['away_score'], 1, 0)
    
    df_teams = pd.concat([df_filtered.rename(columns={'home_team': 'team', 'Win_home': 'Win','Win_away':'Lose', 'home_score' : 'GF', 'away_score': 'GA',}),
                             df_filtered.rename(columns={'away_team': 'team', 'Win_away': 'Win','Win_home':'Lose', 'home_score' : 'GA', 'away_score': 'GF'})], ignore_index=True)
    return(df_teams)

df_teams = separate_wins_loses_ties(df_filtered) 
df_teams=format_date(df_teams)
df_teams = df_teams.sort_values(by=['team', 'date1'])



df_teams['team_changed'] = df_teams['team'] != df_teams['team'].shift(1)
df_teams['streak_start'] = (df_teams['Lose'] == 0) & (df_teams['Lose'].shift(1) != 1)| df_teams['team_changed']

df_teams['streak_decrement'] = (df_teams['streak_start'] == False).astype(int)

df_teams['streak'] = df_teams['streak_decrement'].cumsum().sub(1)

streak_counts = df_teams['streak'].value_counts()
max_values=streak_counts.head(10)

Firsts=[]
Lasts=[]
for i in range (len(max_values)):
    first_index=(df_teams[df_teams['streak'] == max_values.index[i]]['streak'].idxmin())
    last_index=(df_teams[df_teams['streak'] == max_values.index[i]][::-1]['streak'].idxmin())
    Firsts.append(first_index)
    Lasts.append(last_index)
    df_teams.loc[first_index,'Rows']=max_values.iloc[i]
    df_teams.loc[last_index,'Rows']=max_values.iloc[i]


selected_rows = df_teams.loc[Firsts+Lasts]
df_clean=selected_rows[['team','date1','Rows']]
df_clean=df_clean.sort_values(by=['team', 'date1']).reset_index().reset_index()
df_clean['Marker'] = df_clean['level_0'].apply(lambda x: math.trunc(x / 2))
df_clean = df_clean.groupby(['team', 'Marker','Rows'])['date1'].agg(lambda x: 'a'.join(x.dt.strftime('%d-%m-%Y'))).reset_index()
df_clean = df_clean.sort_values(by='Rows', ascending=False).reset_index()
df_clean = df_clean.drop(columns=['Marker','index'])
df_clean['date1'] = df_clean['date1'].str.replace('-', '/').str.replace('a', '-')
df_clean = df_clean[['Rows','team','date1']]


print(df_clean)